In [1]:
import pandas as pd
# from dowhy import gcm
import numpy as np
import matplotlib.pyplot as plt
import gc

import os

In [214]:
node = "stoleman"

In [215]:
#import exp_filtered
data = pd.read_csv('../Datasets/experiments_valid.csv',header=0, index_col=0)
experiments = pd.DataFrame(data)

experiments.tail(10)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
409,891,2024-02-15 19:25:21.9807959,2024-02-15 19:55:21.9832958,validator,random33,19,8,23,4,8,2,0.25,3.00,1708025121,1708026921
412,898,2024-02-15 23:20:24.1114255,2024-02-15 23:50:24.1747705,validator,random5,8,6,16,4,8,2,0.25,0.25,1708039224,1708041024
413,899,2024-02-15 23:53:58.9555104,2024-02-16 00:23:59.0175630,validator,random5,8,6,16,4,8,2,0.25,0.25,1708041238,1708043039
415,901,2024-02-16 01:01:07.9360686,2024-02-16 01:31:07.9411590,validator,random6,16,8,20,6,8,2,0.25,1.00,1708045267,1708047067
416,902,2024-02-16 01:34:42.7924620,2024-02-16 02:04:42.7974999,validator,random6,16,8,20,6,8,2,0.25,1.00,1708047282,1708049082
418,904,2024-02-16 02:41:52.3057095,2024-02-16 03:11:52.3106811,validator,random7,10,8,16,4,8,4,0.25,1.00,1708051312,1708053112
419,905,2024-02-16 03:15:26.8680015,2024-02-16 03:45:26.8705969,validator,random7,10,8,16,4,8,4,0.25,1.00,1708053326,1708055126
421,907,2024-02-16 04:22:35.4565262,2024-02-16 04:52:35.4619071,validator,random8,10,8,16,4,8,2,0.15,1.00,1708057355,1708059155
422,908,2024-02-16 04:56:10.5331406,2024-02-16 05:26:10.5389618,validator,random8,10,8,16,4,8,2,0.15,1.00,1708059370,1708061170
423,909,2024-02-16 05:29:45.3956405,2024-02-16 05:59:45.3984223,validator,reference,8,6,12,4,8,2,0.25,1.00,1708061385,1708063185


In [216]:
messages = pd.DataFrame()

for index, row in experiments.iterrows():
    if os.path.exists('../Datasets/partialTables/mess_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/mess_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_mess = pd.DataFrame(data)
        query_mess = query_mess.drop(columns=["start", "end"])

        messages = pd.concat([query_mess, messages])
        del query_mess
        gc.collect()

    if os.path.exists('../Datasets/partialTables/ihave_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/ihave_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_ihave = pd.DataFrame(data)
        query_ihave["type"] = '17.0'
        query_ihave = query_ihave.drop(columns=["_measurement"])
        
        messages = pd.concat([query_ihave, messages])
        del query_ihave
        gc.collect()
    
    if os.path.exists('../Datasets/partialTables/iwant_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/iwant_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_iwant = pd.DataFrame(data)
        query_iwant["type"] = '16.0'
        query_iwant = query_iwant.drop(columns=["_measurement"])
    
        messages = pd.concat([query_iwant, messages])
        del query_iwant
        gc.collect()
    

messages.head(10)

,_time,type,experiment
0,2024-02-16 05:36:57.845592+00:00,11.0,909
1,2024-02-16 05:36:57.845599+00:00,11.0,909
2,2024-02-16 05:36:57.845603+00:00,11.0,909
3,2024-02-16 05:36:57.845606+00:00,11.0,909
4,2024-02-16 05:36:57.845632+00:00,11.0,909
5,2024-02-16 05:36:57.845635+00:00,11.0,909
6,2024-02-16 05:36:57.845638+00:00,11.0,909
7,2024-02-16 05:36:57.845641+00:00,11.0,909
8,2024-02-16 05:36:57.845650+00:00,11.0,909
9,2024-02-16 05:36:57.845653+00:00,11.0,909


In [217]:
# Transpose the table -> type and count
df = messages#.drop(columns=['identifier'])
df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],df['type'],pd.Grouper(key="_time", freq='5s')])['index'].count().reset_index()


pivoted = pd.pivot_table(by_time,index=["experiment", pd.Grouper(freq="5s", key="_time")], columns="type", values="index", margins=False).fillna(0)
pivoted = pivoted.reset_index().rename_axis(None, axis=1)

pivoted.head(100)

/tmp/ipykernel_3382/2738252698.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,2.0,3.0,11.0,12.0,16.0,17.0
0,390,2023-10-04 18:52:00+00:00,313.0,49.0,0.0,0.0,0.0,0.0
1,390,2023-10-04 18:52:05+00:00,366.0,52.0,0.0,0.0,0.0,55.0
2,390,2023-10-04 18:52:10+00:00,683.0,100.0,0.0,0.0,0.0,55.0
3,390,2023-10-04 18:52:15+00:00,629.0,92.0,0.0,0.0,0.0,28.0
4,390,2023-10-04 18:52:20+00:00,469.0,66.0,0.0,0.0,0.0,54.0
...,...,...,...,...,...,...,...,...
95,390,2023-10-04 18:59:55+00:00,678.0,101.0,0.0,0.0,0.0,56.0
96,390,2023-10-04 19:00:00+00:00,422.0,61.0,0.0,0.0,0.0,28.0
97,390,2023-10-04 19:00:05+00:00,678.0,102.0,0.0,0.0,0.0,55.0
98,390,2023-10-04 19:00:10+00:00,680.0,102.0,0.0,0.0,0.0,56.0


In [218]:
#Rename
eventsTS = pivoted.rename(columns={2.0:'messageDuplicated', 3.0: 'messageReceived', 11.0: 'graft', 12.0: 'prune', '16.0': 'iwant', '17.0':'ihave'})

# eventsTS.to_csv('./events20s.csv')
# eventsTS.head(500)

In [219]:
#Import partial tables and append -> for propagation time

publishMessage = pd.DataFrame()
deliverMessage = pd.DataFrame()

query_mess = pd.DataFrame()
query_pub = pd.DataFrame()

for index, row in experiments.iterrows():
    if os.path.exists('../Datasets/partialTables/mess2_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/mess2_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_mess = pd.DataFrame(data)
        deliverMessage = pd.concat([query_mess, deliverMessage])
        del query_mess

    if os.path.exists('../Datasets/partialTables/pub_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/pub_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_pub = pd.DataFrame(data)
        publishMessage = pd.concat([publishMessage, query_pub])
        del query_pub

    gc.collect()

deliverMessage.head(10)


,_time,messageID,experiment
0,2024-02-16 05:36:57.938178+00:00,ACQIARIg3KUU2uoJavOyzcVNRJLjO/l6ByIRKxWBd/gS56...,909
0,2024-02-16 05:03:27.093290+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,908
1,2024-02-16 05:03:27.104103+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,908
2,2024-02-16 05:03:27.132762+00:00,ACQIARIguLx6PkCG/GbhqGyJR3noZ4kxHLDCpZhJnTh8I0...,908
3,2024-02-16 05:03:27.208576+00:00,ACQIARIgqtBzkLlcGE0XEpfVekpvvEWamsRXGjEL1K5+bI...,908
4,2024-02-16 05:03:27.245836+00:00,ACQIARIg3KUU2uoJavOyzcVNRJLjO/l6ByIRKxWBd/gS56...,908
0,2024-02-16 04:29:49.737597+00:00,ACQIARIguLx6PkCG/GbhqGyJR3noZ4kxHLDCpZhJnTh8I0...,907
1,2024-02-16 04:29:50.069413+00:00,ACQIARIg2E5YMFznHacBOytrj+HcCNQBtpgMqFDEHtnRlQ...,907
2,2024-02-16 04:29:50.172854+00:00,ACQIARIguLx6PkCG/GbhqGyJR3noZ4kxHLDCpZhJnTh8I0...,907
0,2024-02-16 03:22:39.297443+00:00,ACQIARIgqtBzkLlcGE0XEpfVekpvvEWamsRXGjEL1K5+bI...,905


In [220]:
publishMessage['_time'] = pd.to_datetime(publishMessage["_time"], format='mixed')#.tz_localize(None)
deliverMessage['_time'] = pd.to_datetime(deliverMessage["_time"], format='mixed')#.tz_localize(None)

# def calcAverageTime(publish, received, expTime, parameter):
expTime = experiments
publish = publishMessage
received = deliverMessage

publish = publish[['_time', 'messageID','experiment']]
received = received[['_time', 'messageID', 'experiment']]

joined = publish.merge(received, on=['messageID', 'experiment'])
joined['diff'] = ((joined['_time_y'] - joined['_time_x'])/ pd.Timedelta(microseconds=1)).astype(int)
joined = joined.loc[joined["diff"] >= 0].dropna()
joined.head(10)


,_time_x,messageID,experiment,_time_y,diff
0,2023-10-04 18:52:04.485441+00:00,ACQIARIguLx6PkCG/GbhqGyJR3noZ4kxHLDCpZhJnTh8I0...,390,2023-10-04 18:52:04.485710+00:00,269
1,2023-10-04 18:52:04.485734+00:00,ACQIARIguLx6PkCG/GbhqGyJR3noZ4kxHLDCpZhJnTh8I0...,390,2023-10-04 18:52:04.486306+00:00,572
2,2023-10-04 18:52:04.485946+00:00,ACQIARIguLx6PkCG/GbhqGyJR3noZ4kxHLDCpZhJnTh8I0...,390,2023-10-04 18:52:04.486358+00:00,412
3,2023-10-04 18:52:04.486170+00:00,ACQIARIguLx6PkCG/GbhqGyJR3noZ4kxHLDCpZhJnTh8I0...,390,2023-10-04 18:52:04.486412+00:00,242
4,2023-10-04 18:52:04.486466+00:00,ACQIARIguLx6PkCG/GbhqGyJR3noZ4kxHLDCpZhJnTh8I0...,390,2023-10-04 18:52:04.487121+00:00,655
5,2023-10-04 18:52:07.492859+00:00,ACQIARIguLx6PkCG/GbhqGyJR3noZ4kxHLDCpZhJnTh8I0...,390,2023-10-04 18:52:07.493178+00:00,319
6,2023-10-04 18:52:07.493781+00:00,ACQIARIguLx6PkCG/GbhqGyJR3noZ4kxHLDCpZhJnTh8I0...,390,2023-10-04 18:52:07.494001+00:00,220
7,2023-10-04 18:52:07.494588+00:00,ACQIARIguLx6PkCG/GbhqGyJR3noZ4kxHLDCpZhJnTh8I0...,390,2023-10-04 18:52:07.494822+00:00,234
8,2023-10-04 18:52:07.495617+00:00,ACQIARIguLx6PkCG/GbhqGyJR3noZ4kxHLDCpZhJnTh8I0...,390,2023-10-04 18:52:07.495835+00:00,218
9,2023-10-04 18:52:07.495935+00:00,ACQIARIguLx6PkCG/GbhqGyJR3noZ4kxHLDCpZhJnTh8I0...,390,2023-10-04 18:52:07.496119+00:00,184


In [221]:
df = joined.drop(columns=['_time_x', 'messageID']).rename(columns={'_time_y' : '_time'})

df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],pd.Grouper(key="_time", freq='5s')])['diff'].agg('mean').reset_index()
by_time.head(100)

/tmp/ipykernel_3382/2407133802.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,diff
0,390,2023-10-04 18:52:00+00:00,430.0
1,390,2023-10-04 18:52:05+00:00,235.0
2,390,2023-10-04 18:52:10+00:00,581.7
3,390,2023-10-04 18:52:15+00:00,1102.7
4,390,2023-10-04 18:52:20+00:00,290.8
...,...,...,...
95,390,2023-10-04 18:59:55+00:00,381.3
96,390,2023-10-04 19:00:00+00:00,386.8
97,390,2023-10-04 19:00:05+00:00,311.4
98,390,2023-10-04 19:00:10+00:00,304.2


In [222]:
prop = by_time.rename(columns={'diff':'propagationTime'})

prop = prop.merge(eventsTS, on=['experiment', '_time'], how='outer')
prop.head(100)

,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave
0,390,2023-10-04 18:52:00+00:00,430.0,313.0,49.0,0.0,0.0,0.0,0.0
1,390,2023-10-04 18:52:05+00:00,235.0,366.0,52.0,0.0,0.0,0.0,55.0
2,390,2023-10-04 18:52:10+00:00,581.7,683.0,100.0,0.0,0.0,0.0,55.0
3,390,2023-10-04 18:52:15+00:00,1102.7,629.0,92.0,0.0,0.0,0.0,28.0
4,390,2023-10-04 18:52:20+00:00,290.8,469.0,66.0,0.0,0.0,0.0,54.0
...,...,...,...,...,...,...,...,...,...
95,390,2023-10-04 18:59:55+00:00,381.3,678.0,101.0,0.0,0.0,0.0,56.0
96,390,2023-10-04 19:00:00+00:00,386.8,422.0,61.0,0.0,0.0,0.0,28.0
97,390,2023-10-04 19:00:05+00:00,311.4,678.0,102.0,0.0,0.0,0.0,55.0
98,390,2023-10-04 19:00:10+00:00,304.2,680.0,102.0,0.0,0.0,0.0,56.0


In [223]:
rpc = pd.DataFrame()

for idx, exec in experiments.iterrows():
    data = pd.read_csv('../Datasets/partialTables/rpc_'+node+'_'+str(exec['experiment'])+'.csv', header=0,  index_col=0)
    query_rpc = pd.DataFrame(data)
        
    rpc = pd.concat([rpc,query_rpc])

    del query_rpc
    gc.collect()
        

rpc.head(10)

,_time,_measurement,experiment,start,end
0,2023-10-04 18:52:04.135293+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
1,2023-10-04 18:52:04.135840+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
2,2023-10-04 18:52:04.135852+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
3,2023-10-04 18:52:04.135938+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
4,2023-10-04 18:52:04.136030+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
5,2023-10-04 18:52:04.136045+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
6,2023-10-04 18:52:04.136052+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
7,2023-10-04 18:52:04.136066+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
8,2023-10-04 18:52:04.136149+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
9,2023-10-04 18:52:04.136402+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518


In [224]:
rpc = rpc.drop(columns=['start', 'end'])
                       
rpc.reset_index(inplace=True)
rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)
by_time = rpc.groupby([rpc['experiment'],pd.Grouper(key="_time", freq='5s')])['index'].agg('count').reset_index()
# by_time.head(100)

rpc=by_time.rename(columns={'index':'rpc'})

/tmp/ipykernel_3382/1580870231.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)


In [225]:
message=received

message.reset_index(inplace=True)
message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)
by_time = message.groupby([message['experiment'],pd.Grouper(key="_time", freq='5s')])['messageID'].agg('count').reset_index()
message = by_time

message.head(10)

/tmp/ipykernel_3382/1432395461.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,messageID
0,390,2023-10-04 18:52:00+00:00,49
1,390,2023-10-04 18:52:05+00:00,52
2,390,2023-10-04 18:52:10+00:00,100
3,390,2023-10-04 18:52:15+00:00,92
4,390,2023-10-04 18:52:20+00:00,66
5,390,2023-10-04 18:52:25+00:00,100
6,390,2023-10-04 18:52:30+00:00,88
7,390,2023-10-04 18:52:35+00:00,89
8,390,2023-10-04 18:52:40+00:00,102
9,390,2023-10-04 18:52:45+00:00,71


In [226]:
message = message.rename(columns={'messageID':'count'})
rpc = rpc.rename(columns={'rpc': 'count'})
total_bandwidth = pd.concat([rpc, message])

total_bandwidth.reset_index(inplace=True)
total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)
by_time = total_bandwidth.groupby([total_bandwidth['experiment'],pd.Grouper(key="_time", freq='5s')])['count'].agg('sum').reset_index()
by_time.head(100)

/tmp/ipykernel_3382/448925867.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,count
0,390,2023-10-04 18:52:00+00:00,407
1,390,2023-10-04 18:52:05+00:00,492
2,390,2023-10-04 18:52:10+00:00,901
3,390,2023-10-04 18:52:15+00:00,818
4,390,2023-10-04 18:52:20+00:00,623
...,...,...,...
95,390,2023-10-04 18:59:55+00:00,899
96,390,2023-10-04 19:00:00+00:00,553
97,390,2023-10-04 19:00:05+00:00,901
98,390,2023-10-04 19:00:10+00:00,902


In [227]:
by_time['totalBandwidth'] = by_time['count']

totalBandwidth = by_time.drop(columns=['count'])
totalBandwidth.head(10)

,experiment,_time,totalBandwidth
0,390,2023-10-04 18:52:00+00:00,407
1,390,2023-10-04 18:52:05+00:00,492
2,390,2023-10-04 18:52:10+00:00,901
3,390,2023-10-04 18:52:15+00:00,818
4,390,2023-10-04 18:52:20+00:00,623
5,390,2023-10-04 18:52:25+00:00,905
6,390,2023-10-04 18:52:30+00:00,766
7,390,2023-10-04 18:52:35+00:00,845
8,390,2023-10-04 18:52:40+00:00,919
9,390,2023-10-04 18:52:45+00:00,637


In [228]:
rpc.head(10)

,experiment,_time,count
0,390,2023-10-04 18:52:00+00:00,358
1,390,2023-10-04 18:52:05+00:00,440
2,390,2023-10-04 18:52:10+00:00,801
3,390,2023-10-04 18:52:15+00:00,726
4,390,2023-10-04 18:52:20+00:00,557
5,390,2023-10-04 18:52:25+00:00,805
6,390,2023-10-04 18:52:30+00:00,678
7,390,2023-10-04 18:52:35+00:00,756
8,390,2023-10-04 18:52:40+00:00,817
9,390,2023-10-04 18:52:45+00:00,566


In [229]:
#gossip bandwidth
gossipBandwidth = rpc
gossipBandwidth['gossipReceived'] = gossipBandwidth['count']
gossipBandwidth['gossipBandwidth'] = gossipBandwidth['count']/5
gossipBandwidth = gossipBandwidth.drop(columns=['count'])

gossipBandwidth.head(100)

,experiment,_time,gossipReceived,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,358,71.6
1,390,2023-10-04 18:52:05+00:00,440,88.0
2,390,2023-10-04 18:52:10+00:00,801,160.2
3,390,2023-10-04 18:52:15+00:00,726,145.2
4,390,2023-10-04 18:52:20+00:00,557,111.4
...,...,...,...,...
95,390,2023-10-04 18:59:55+00:00,798,159.6
96,390,2023-10-04 19:00:00+00:00,492,98.4
97,390,2023-10-04 19:00:05+00:00,799,159.8
98,390,2023-10-04 19:00:10+00:00,800,160.0


In [230]:
#MessageBandwidth

messageBandwidth = message
messageBandwidth['messageBandwidth'] = messageBandwidth['count']/5
messageBandwidth = messageBandwidth.drop(columns=['count'])

messageBandwidth.head(10)

,experiment,_time,messageBandwidth
0,390,2023-10-04 18:52:00+00:00,9.8
1,390,2023-10-04 18:52:05+00:00,10.4
2,390,2023-10-04 18:52:10+00:00,20.0
3,390,2023-10-04 18:52:15+00:00,18.4
4,390,2023-10-04 18:52:20+00:00,13.2
5,390,2023-10-04 18:52:25+00:00,20.0
6,390,2023-10-04 18:52:30+00:00,17.6
7,390,2023-10-04 18:52:35+00:00,17.8
8,390,2023-10-04 18:52:40+00:00,20.4
9,390,2023-10-04 18:52:45+00:00,14.2


In [231]:
prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)

messBand = prop.merge(messageBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

messBand.head(500)

/tmp/ipykernel_3382/2873079763.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_3382/2873079763.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth
0,390,2023-10-04 18:52:00+00:00,430.0,313.0,49.0,0.0,0.0,0.0,0.0,9.8
1,390,2023-10-04 18:52:05+00:00,235.0,366.0,52.0,0.0,0.0,0.0,55.0,10.4
2,390,2023-10-04 18:52:10+00:00,581.7,683.0,100.0,0.0,0.0,0.0,55.0,20.0
3,390,2023-10-04 18:52:15+00:00,1102.7,629.0,92.0,0.0,0.0,0.0,28.0,18.4
4,390,2023-10-04 18:52:20+00:00,290.8,469.0,66.0,0.0,0.0,0.0,54.0,13.2
...,...,...,...,...,...,...,...,...,...,...
495,404,2023-10-05 02:59:05+00:00,322.6,292.0,66.0,0.0,0.0,0.0,25.0,13.2
496,404,2023-10-05 02:59:10+00:00,381.6,299.0,66.0,0.0,0.0,0.0,27.0,13.2
497,404,2023-10-05 02:59:15+00:00,403.8,180.0,39.0,0.0,0.0,0.0,14.0,7.8
498,404,2023-10-05 02:59:20+00:00,281.9,271.0,60.0,0.0,0.0,0.0,27.0,12.0


In [232]:
messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

totalBand = messBand.merge(totalBandwidth, on=['experiment', '_time'], how='outer').fillna(0)
totalBand['totalBandwidth'] = (totalBand['totalBandwidth']+totalBand['messageDuplicated'])/5

totalBand.head(500)

/tmp/ipykernel_3382/511213079.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_3382/511213079.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth
0,390,2023-10-04 18:52:00+00:00,430.0,313.0,49.0,0.0,0.0,0.0,0.0,9.8,144.0
1,390,2023-10-04 18:52:05+00:00,235.0,366.0,52.0,0.0,0.0,0.0,55.0,10.4,171.6
2,390,2023-10-04 18:52:10+00:00,581.7,683.0,100.0,0.0,0.0,0.0,55.0,20.0,316.8
3,390,2023-10-04 18:52:15+00:00,1102.7,629.0,92.0,0.0,0.0,0.0,28.0,18.4,289.4
4,390,2023-10-04 18:52:20+00:00,290.8,469.0,66.0,0.0,0.0,0.0,54.0,13.2,218.4
...,...,...,...,...,...,...,...,...,...,...,...
495,404,2023-10-05 02:59:05+00:00,322.6,292.0,66.0,0.0,0.0,0.0,25.0,13.2,143.6
496,404,2023-10-05 02:59:10+00:00,381.6,299.0,66.0,0.0,0.0,0.0,27.0,13.2,146.8
497,404,2023-10-05 02:59:15+00:00,403.8,180.0,39.0,0.0,0.0,0.0,14.0,7.8,88.2
498,404,2023-10-05 02:59:20+00:00,281.9,271.0,60.0,0.0,0.0,0.0,27.0,12.0,133.2


In [233]:
totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

gossipBand = totalBand.merge(gossipBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

gossipBand.head(500)

/tmp/ipykernel_3382/187803194.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_3382/187803194.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipReceived,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,430.0,313.0,49.0,0.0,0.0,0.0,0.0,9.8,144.0,358.0,71.6
1,390,2023-10-04 18:52:05+00:00,235.0,366.0,52.0,0.0,0.0,0.0,55.0,10.4,171.6,440.0,88.0
2,390,2023-10-04 18:52:10+00:00,581.7,683.0,100.0,0.0,0.0,0.0,55.0,20.0,316.8,801.0,160.2
3,390,2023-10-04 18:52:15+00:00,1102.7,629.0,92.0,0.0,0.0,0.0,28.0,18.4,289.4,726.0,145.2
4,390,2023-10-04 18:52:20+00:00,290.8,469.0,66.0,0.0,0.0,0.0,54.0,13.2,218.4,557.0,111.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,404,2023-10-05 02:59:05+00:00,322.6,292.0,66.0,0.0,0.0,0.0,25.0,13.2,143.6,360.0,72.0
496,404,2023-10-05 02:59:10+00:00,381.6,299.0,66.0,0.0,0.0,0.0,27.0,13.2,146.8,369.0,73.8
497,404,2023-10-05 02:59:15+00:00,403.8,180.0,39.0,0.0,0.0,0.0,14.0,7.8,88.2,222.0,44.4
498,404,2023-10-05 02:59:20+00:00,281.9,271.0,60.0,0.0,0.0,0.0,27.0,12.0,133.2,335.0,67.0


In [234]:
consDiscovery = gossipBand.rename(columns={'messageDuplicated' : 'messageOverhead'})#.drop(columns=['bandwidth'])
consDiscovery.head(10)

,experiment,_time,propagationTime,messageOverhead,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipReceived,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,430.0,313.0,49.0,0.0,0.0,0.0,0.0,9.8,144.0,358.0,71.6
1,390,2023-10-04 18:52:05+00:00,235.0,366.0,52.0,0.0,0.0,0.0,55.0,10.4,171.6,440.0,88.0
2,390,2023-10-04 18:52:10+00:00,581.7,683.0,100.0,0.0,0.0,0.0,55.0,20.0,316.8,801.0,160.2
3,390,2023-10-04 18:52:15+00:00,1102.7,629.0,92.0,0.0,0.0,0.0,28.0,18.4,289.4,726.0,145.2
4,390,2023-10-04 18:52:20+00:00,290.8,469.0,66.0,0.0,0.0,0.0,54.0,13.2,218.4,557.0,111.4
5,390,2023-10-04 18:52:25+00:00,287.8,687.0,100.0,0.0,0.0,0.0,55.0,20.0,318.4,805.0,161.0
6,390,2023-10-04 18:52:30+00:00,391.2,586.0,88.0,0.0,0.0,0.0,27.0,17.6,270.4,678.0,135.6
7,390,2023-10-04 18:52:35+00:00,430.2,645.0,89.0,0.0,0.0,0.0,54.0,17.8,298.0,756.0,151.2
8,390,2023-10-04 18:52:40+00:00,285.2,695.0,102.0,0.0,0.0,0.0,56.0,20.4,322.8,817.0,163.4
9,390,2023-10-04 18:52:45+00:00,363.4,486.0,71.0,0.0,0.0,0.0,28.0,14.2,224.6,566.0,113.2


In [235]:
consFinal = consDiscovery.merge(experiments, on='experiment')
consFinal = consFinal.drop(columns=['_time', 'start', 'end', 'startUnix', 'endUnix', 'parameter'])#, 'originalStart', 'originalEnd']) 

consFinal.to_csv('../Datasets/1node/raw/consolidated5s_'+node+'.csv')
consFinal.head(10)


,experiment,propagationTime,messageOverhead,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,...,gossipBandwidth,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval
0,390,430.0,313.0,49.0,0.0,0.0,0.0,0.0,9.8,144.0,...,71.6,unl,8,6,12,4,8,2,0.25,1.0
1,390,235.0,366.0,52.0,0.0,0.0,0.0,55.0,10.4,171.6,...,88.0,unl,8,6,12,4,8,2,0.25,1.0
2,390,581.7,683.0,100.0,0.0,0.0,0.0,55.0,20.0,316.8,...,160.2,unl,8,6,12,4,8,2,0.25,1.0
3,390,1102.7,629.0,92.0,0.0,0.0,0.0,28.0,18.4,289.4,...,145.2,unl,8,6,12,4,8,2,0.25,1.0
4,390,290.8,469.0,66.0,0.0,0.0,0.0,54.0,13.2,218.4,...,111.4,unl,8,6,12,4,8,2,0.25,1.0
5,390,287.8,687.0,100.0,0.0,0.0,0.0,55.0,20.0,318.4,...,161.0,unl,8,6,12,4,8,2,0.25,1.0
6,390,391.2,586.0,88.0,0.0,0.0,0.0,27.0,17.6,270.4,...,135.6,unl,8,6,12,4,8,2,0.25,1.0
7,390,430.2,645.0,89.0,0.0,0.0,0.0,54.0,17.8,298.0,...,151.2,unl,8,6,12,4,8,2,0.25,1.0
8,390,285.2,695.0,102.0,0.0,0.0,0.0,56.0,20.4,322.8,...,163.4,unl,8,6,12,4,8,2,0.25,1.0
9,390,363.4,486.0,71.0,0.0,0.0,0.0,28.0,14.2,224.6,...,113.2,unl,8,6,12,4,8,2,0.25,1.0
